In [1]:
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
os.chdir("../src/")
%pwd

'/home/jovyan/nfs-home/scalable_ml_pipelines/src'

In [3]:
import socket

LOCAL_IP = socket.gethostbyname(socket.gethostname())

In [4]:
name_space = "eabraham-373705"

# Master node
kubernetes_master_url = "k8s://https://10.32.7.103:6443"

# Resource settings
driver_cores = "8"
executor_cores = "8"
driver_memory = "30g"
executor_memory = "30g"
executor_memory_overhead = "2g"

# These are the limits
cpu_limit = "3"  # 12 cores
memory_limit = "32g"  # Upto 32 GB
executor_limit = "8"

In [5]:
from pyspark.sql import SparkSession

APP_NAME = 'scalables_executor'


spark = SparkSession\
    .builder\
    .appName(APP_NAME)\
    .master(kubernetes_master_url)\
    .config("spark.driver.host", LOCAL_IP)\
    .config("spark.driver.bindAddress", "0.0.0.0")\
    .config("spark.executor.instances", "2")\
    .config("spark.executor.cores", executor_cores)\
    .config("spark.executor.memory", executor_memory)\
    .config("spark.memory.fraction", "0.8")\
    .config("spark.memory.storageFraction", "0.2")\
    .config("spark.kubernetes.executor.limit.cores", executor_limit)\
    .config("spark.kubernetes.namespace", name_space)\
    .config("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")\
    .config("spark.kubernetes.driver.label.appname", APP_NAME)\
    .config("spark.kubernetes.executor.label.appname", APP_NAME)\
    .config("spark.kubernetes.executor.deleteOnTermination", "false") \
    .config("spark.kubernetes.container.image.pullPolicy", "Always") \
    .config("spark.kubernetes.container.image", "node03.st:5000/pyspark-hdfs-jupyter:eabraham-373705-v4-executor")\
    .config("spark.local.dir", "/tmp/spark")\
    .config("spark.kubernetes.driver.volumes.emptyDir.spark-local-dir-tmp-spark.mount.path", "/tmp/spark")\
    .config("spark.kubernetes.driver.volumes.emptyDir.spark-local-dir-tmp-spark.mount.readOnly", "false")\
    .config("spark.kubernetes.executor.volumes.emptyDir.spark-local-dir-tmp-spark.mount.path", "/tmp/spark")\
    .config("spark.kubernetes.executor.volumes.emptyDir.spark-local-dir-tmp-spark.mount.readOnly", "false")\
    .getOrCreate()


23/12/17 14:49:00 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/17 14:49:00 WARN spark.SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/12/17 14:49:01 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


KeyboardInterrupt: 

In [ ]:
spark

In [ ]:
from us_used_cars_ml_pipeline.config.configuration import ConfigurationManager
from us_used_cars_ml_pipeline.components.feature_selection import FeatureSelection

config_manager = ConfigurationManager()
feature_selection_config = config_manager.get_feature_selection_config()

feature_selection = FeatureSelection(feature_selection_config)

In [ ]:
feature_selection_config

In [ ]:
%%time
feature_selection.run_stage(spark)

# Testing parts separately

In [9]:
%%time
# Importance values calculation
feature_selection.run_stage(spark, include_search=False)

[2023-12-17 13:54:35,790: 291: us_used_cars_ml_pipeline_logger: INFO: feature_selection:  Prepared data has been read]
[2023-12-17 13:54:36,159: 295: us_used_cars_ml_pipeline_logger: INFO: feature_selection:  Data has been splitted into training and testing sets]
[2023-12-17 13:54:36,368: 301: us_used_cars_ml_pipeline_logger: INFO: feature_selection:  Features has been converted to vectors]


  0%|          | 0/3 [00:00<?, ?it/s]

23/12/17 13:54:54 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/12/17 13:54:54 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


[2023-12-17 13:55:02,665: 308: us_used_cars_ml_pipeline_logger: INFO: feature_selection:  Model LinearRegression has been trained on the training set]


[2023-12-17 13:56:29,508: 308: us_used_cars_ml_pipeline_logger: INFO: feature_selection:  Model RandomForestRegressor has been trained on the training set]


...

[2023-12-17 14:04:31,652: 66: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file saved at: us_used_cars_ml_pipeline/tests/importances/GBTRegressor.yaml]


23/12/17 14:04:43 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1771.7 KiB
23/12/17 14:04:44 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1772.8 KiB


[2023-12-17 14:04:45,402: 66: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file saved at: us_used_cars_ml_pipeline/tests/importances/GBTRegressor.yaml]


23/12/17 14:04:56 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1771.7 KiB
23/12/17 14:04:58 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1772.8 KiB


[2023-12-17 14:04:59,065: 66: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file saved at: us_used_cars_ml_pipeline/tests/importances/GBTRegressor.yaml]
[2023-12-17 14:04:59,068: 340: us_used_cars_ml_pipeline_logger: INFO: feature_selection:  Importances have been calculated for GBTRegressor]
[2023-12-17 14:04:59,070: 400: us_used_cars_ml_pipeline_logger: INFO: feature_selection:  All scores have been calculated and saved]
CPU times: user 1min 51s, sys: 3.83 s, total: 1min 54s
Wall time: 10min 27s


In [9]:
%%time
# Estimating top features sets
feature_selection.run_stage(spark, include_importance_calculation=False)

[2023-12-17 14:20:23,252: 291: us_used_cars_ml_pipeline_logger: INFO: feature_selection:  Prepared data has been read]
[2023-12-17 14:20:23,643: 295: us_used_cars_ml_pipeline_logger: INFO: feature_selection:  Data has been splitted into training and testing sets]


  0%|          | 0/3 [00:00<?, ?it/s]

[2023-12-17 14:20:23,677: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/tests/importances/LinearRegression.yaml loaded successfully]
[2023-12-17 14:20:23,699: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/tests/importances/RandomForestRegressor.yaml loaded successfully]
[2023-12-17 14:20:23,720: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/tests/importances/GBTRegressor.yaml loaded successfully]
[2023-12-17 14:20:23,724: 348: us_used_cars_ml_pipeline_logger: INFO: feature_selection:  Importance calculation part has been skipped. Using stored values]


[2023-12-17 14:20:32,695: 358: us_used_cars_ml_pipeline_logger: INFO: feature_selection:  Training set has been cached]


[2023-12-17 14:20:36,699: 362: us_used_cars_ml_pipeline_logger: INFO: feature_selection:  Evaluation set has been cached]


...

[2023-12-17 14:40:15,155: 66: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file saved at: us_used_cars_ml_pipeline/tests/feature_selection_scores/GBTRegressor.yaml]


[2023-12-17 14:40:38,808: 66: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file saved at: us_used_cars_ml_pipeline/tests/feature_selection_scores/GBTRegressor.yaml]


[2023-12-17 14:41:00,946: 66: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file saved at: us_used_cars_ml_pipeline/tests/feature_selection_scores/GBTRegressor.yaml]


[2023-12-17 14:41:22,304: 66: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file saved at: us_used_cars_ml_pipeline/tests/feature_selection_scores/GBTRegressor.yaml]


[2023-12-17 14:41:42,892: 66: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file saved at: us_used_cars_ml_pipeline/tests/feature_selection_scores/GBTRegressor.yaml]
[2023-12-17 14:42:02,924: 66: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file saved at: us_used_cars_ml_pipeline/tests/feature_selection_scores/GBTRegressor.yaml]
[2023-12-17 14:42:02,927: 408: us_used_cars_ml_pipeline_logger: INFO: feature_selection:  All scores have been calculated and saved]
CPU times: user 4.45 s, sys: 2.23 s, total: 6.68 s
Wall time: 21min 43s
